# Importando dados e bibliotecas

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [ ]:
# Reading Data

df = pd.read_csv("../input/nlp-getting-started/train.csv")
df.info()

In [ ]:
df.head(5)

# Criando Classificadores

In [ ]:
vect = CountVectorizer(analyzer='word')
transf = TfidfTransformer()
clf_Gaussian = GaussianNB()
clf_Bernoulli = BernoulliNB()
clf_Tree = DecisionTreeClassifier()

In [ ]:
df_X = vect.fit_transform(df.text)
X = transf.fit_transform(df_X)
Y = df.target

In [ ]:
df.text[0]

In [ ]:
print(df_X.toarray()[0].sum())

In [ ]:
print(X.toarray()[0].sum())

# Taxa de Acerto Base

Antes de fazer qualquer classificação, vamos verificar a taxa base de acerto. Essa taxa remete ao quanto de acerto teríamos se chutássemos um valor sempre verdadeiro ou sempre falso.

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X, Y)
dummy_clf.score(X, Y)

Um algoritmo bom tem de acertar no mínimo 57% dos casos.

# Iniciando Classificações

Vamos separar o dataset em teste e treino com 80% e 20% respectivamente. E treinar os modelos:

* Naive Bayes Gaussiano
* Naive Bayes Bernoulli
* Árvore de Decisão

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), Y, test_size=0.2, random_state=0)

In [ ]:
model_gaussian = clf_Gaussian.fit(X_train,y_train)
model_bernoulli = clf_Bernoulli.fit(X_train,y_train)
model_tree = clf_Tree.fit(X_train,y_train)

In [ ]:
print("Gaussian: ",model_gaussian.score(X_test,y_test))
print("Bernoulli: ",model_bernoulli.score(X_test,y_test))
print("Decision Tree: ",model_tree.score(X_test,y_test))

# Otimização do Bernoulli

Vamos testar um pipeline com o classificador Bernoulli (não utilizarei a função do sklearn para criar um pipeline, farei o passo a passo)

In [ ]:
# Iniciando Cross_Validation com K-Fold = 5

score_CV = cross_val_score(clf_Bernoulli, X, Y, cv=5)
score_CV

In [ ]:
print("%0.2f de precisão com desvio padrão de %0.2f" % (score_CV.mean(), score_CV.std()))

In [ ]:
clf_Bernoulli.get_params()

In [ ]:
# Otimizando Hiperparâmetros do Classificador
parameters = {
    'alpha' : (0.1,0.5,0.6,0.7,0.8,0.9,1,1.1,1.2,1.3,1.4,1.5,2), # Testando parâmetros alpha de 0.1 a 2
    'fit_prior': (True,False)
}

clf = GridSearchCV(clf_Bernoulli, parameters, cv=5, return_train_score=True)
clf.fit(X,Y)

In [ ]:
clf.best_params_

In [ ]:
# Parametrizando um novo classificador e verificando com CV

clf_Bernoulli_novo = BernoulliNB(alpha= 1.5, fit_prior=False)
score_CV = cross_val_score(clf_Bernoulli_novo, X, Y, cv=5)
print("%0.2f de precisão com desvio padrão de %0.2f" % (score_CV.mean(), score_CV.std()))

Tivemos uma melhoria de apenas 1% na precisão do classificador. Vamos tentar melhorar outro algoritmo (Árvore de Decisão).

In [ ]:
score_CV = cross_val_score(clf_Tree, X, Y, cv=5)
print("%0.2f de precisão com desvio padrão de %0.2f" % (score_CV.mean(), score_CV.std()))

In [ ]:
# Otimizando Hiperparâmetros do Classificador
parameters = {
    'criterion': ('gini', 'entropy'),
    'splitter': ('best','random'),
    'max_depth': (3,5,7,10),
    'random_state' : [0],
}

clf = GridSearchCV(clf_Tree, parameters, cv=5, return_train_score=True)
clf.fit(X,Y)
clf.best_params_

In [ ]:
# Parametrizando um novo classificador e verificando com CV

clf_tree_novo = DecisionTreeClassifier(criterion='gini',max_depth=7,random_state=0,splitter="random")
score_CV = cross_val_score(clf_tree_novo, X, Y, cv=5)
print("%0.2f de precisão com desvio padrão de %0.2f" % (score_CV.mean(), score_CV.std()))

O algoritmo melhorou cerca de 3% em relação ao classificador em "default", contudo, é cerca de 10% menos preciso do que o algoritmo de Bernoulli. Vamos explorar outros conceitos de métricas.

# Métricas 

Vamos explorar duas métricas (matriz de confusão e Curva ROC, do inglês Receiver Operating Characteristic), utilizaremos o classificador otimizado de Bernoulli para fazer estas visualizações e análises.

## Matriz de Confusão

In [ ]:
model = clf_Bernoulli_novo.fit(X_train, y_train)
y_predict = clf_Bernoulli_novo.predict(X_test)

In [ ]:
matriz_confusao = metrics.confusion_matrix(y_test,y_predict,labels=clf_Bernoulli_novo.classes_)
mc_perc = matriz_confusao*100/y_predict.shape[0]

In [ ]:
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=mc_perc,display_labels=clf_Bernoulli_novo.classes_)
disp.plot()
plt.savefig("cm.png")
plt.show()

Neste ponto, podemos realizar a seguinte visualização em termos percentuais:

* 54% dos casos são verdadeiros negativos, ou seja, o algoritmo sinaliza que não há um desastre e acerta;
* 26% dos casos são verdadeiros positivos, isto é, o algoritmo sinaliza que há uma desastre e acerta;
* 4.6% dos casos são falsos positivos, o algoritmo sinaliza que está ocorrendo um desastre mas na verdade não está;
* Por fim, 16% são falsos negativos e esta é a parte mais preocupante, pois o algoritmo não identifica que o tweet é de um desastre, quando na verdade, está acontecendo algo.


## Curva ROC

In [ ]:
model.predict_proba(X_test) # Retorna a probabilidade de amostras para cada classe

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(X_test)[:,1])
roc_auc = metrics.auc(fpr, tpr)
display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='Bernoulli NB')
display.plot(color='Red')
plt.plot([0,1],[0,1],color = 'black')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.savefig("ROC.png")
plt.show()

Aqui, podemos dizer que a Curva ROC é uma relação entre as taxas de falsos positivos e verdadeiros positivos. Para análise, podemos ver que a curva tem um perfil acima da linha de taxa base, o algoritmo tem uma boa funcionalidade e sua AUC (Area Under the Curve) é próxima de 1 (valor máximo para um algoritmo que tem uma baixa taxa de falsos positivos.

Importante salientar que estes dados poderiam ser melhor tratados com algoritmos mais refinados e um trabalho melhor na parte de NLP. Mas aqui é um trabalho de pipeline sem a parte de EDA, primordial para iniciar um trabalho de análise.

# Submissão dos Resultados

Exportando valores para submissão

In [ ]:
df_testing = pd.read_csv("../input/nlp-getting-started/test.csv")
df_X_testing = vect.transform(df_testing.text)
X_testing = transf.transform(df_X_testing)

In [ ]:
df_testing

In [ ]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = model.predict(X_testing)
sample_submission.to_csv('submission.csv', index=False)

Fontes:
* https://scikit-learn.org/
* https://medium.com/kunumi/m%C3%A9tricas-de-avalia%C3%A7%C3%A3o-em-machine-learning-classifica%C3%A7%C3%A3o-49340dcdb198
